The code in `pyldl.applications.emphasis_selection` is basically an unofficial implementation of the paper "Learning Emphasis Selection for Written Text in Visual Media from Crowd-Sourced Label Distributions" (*ACL* 2019).

In [1]:
from sklearn.model_selection import train_test_split
import pyldl.applications.emphasis_selection as es

In [2]:
from pyldl.utils import LDLEarlyStopping

In [3]:
import keras

In [4]:
words, freqs = es.load_semeval2020('./SemEval2020_Task10_Emphasis_Selection/')

In [5]:
es.visualization(words[:5])

In [6]:
words_train, words_test, freqs_train, freqs_test = \
train_test_split(words, freqs, test_size=0.1, random_state=0)

In [7]:
X_train, y_train, tokenizer, maxlen = es.preprocessing(words_train, freqs_train)

In [8]:
embeddings_matrix = es.load_glove('./glove.6B/', tokenizer, embedding_dim=100)

In [9]:
model = es.DL_BiLSTM(tokenizer, embeddings_matrix)

In [10]:
val = int(X_train.shape[0] * 0.9)

In [11]:
model.fit(X_train[:val], y_train[:val], verbose=1, optimizer=keras.optimizers.Adam(5e-4),
          X_val=X_train[val:], y_val=y_train[val:], callbacks=[LDLEarlyStopping(monitor='loss')])

I0000 00:00:1717575503.082867  245797 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 213/1000 [=====>........................] - 19s 74ms/step - loss: 344.5778
Epoch 213: early stopping (best loss: 343.5492248535156).


In [12]:
X_test, y_test = es.preprocessing(words_test, freqs_test, tokenizer, maxlen)

In [13]:
y_pred = model.predict(X_test)

In [14]:
es.visualization(words_test[:5], y_pred[:5])

In [15]:
es.visualization(words_test[:5], y_test[:5])